In [1]:
# !pip install markdownify

In [2]:
import requests
from typing import Dict, Any, Optional, List
from markdownify import markdownify 

In [23]:
def _search_stackoverflow_for_relevant_questions(tag: str, query: str, num_results: int = 5) -> Optional[List[Tuple[str, str]]]:
    """
    Search Stack Overflow for questions based on a specified tag and query.

    Args:
        tag (str): The tag to search for.
        query (str): The query to search for in the title of questions.
        num_results (int, optional): The number of results to return. Defaults to 5.

    Returns:
        Optional[List[Tuple[str, str]]]: A list of tuples containing the question body and accepted answer body,
                                           or None if an error occurs during the request.
    """
    # Stack Exchange API endpoint for searching questions
    url = "https://api.stackexchange.com/2.3/search"
    
    # Parameters for the API request
    params = {
        "order": "desc",
        "sort": "relevance",
        "tagged": tag,  # specify the tag
        "intitle": query,  # specify the query in title
        "site": "stackoverflow",
        "pagesize": num_results,  # specify the number of results
        "answers": 1,  # get questions with at least 1 answer
        "accepted": True,  # get questions with accepted answers
        "filter": "withbody"  # include answer body in response
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
        
        results = []
        # Extract relevant information for each question
        for item in data.get("items", []):
            question_body = markdownify(item["body"], strip=["a", "blockquote"]).rstrip("\n")
            
            # Fetch accepted answer if available
            if "accepted_answer_id" in item:
                answer_id = item["accepted_answer_id"]
                answer_url = f"https://api.stackexchange.com/2.3/answers/{answer_id}"
                answer_params = {
                    "site": "stackoverflow",
                    "filter": "withbody"  # include answer body in response
                }
                answer_response = requests.get(answer_url, params=answer_params)
                answer_data = answer_response.json()
                if "items" in answer_data and len(answer_data["items"]) > 0:
                    # Check if "body" exists and is not empty in the answer data
                    if "body" in answer_data["items"][0] and answer_data["items"][0]["body"].strip():
                        answer_body = markdownify(answer_data["items"][0]["body"], strip=["a", "blockquote"]).rstrip("\n")
                        results.append((question_body, answer_body))
        
        return results
        
    except requests.RequestException as e:
        print("Error fetching data:", e)
        return None

In [28]:
def format_stackoferflow_questions(input_questions: List[Tuple[str, str]]) -> str:
    formatted_result = ""
    for result in input_questions:
        formatted_result += f"---------------\nQuestion:\n{result[0]}\n--------------- \nAccepted Answer:\n{result[1]}"

    return formatted_result

In [24]:
# Example usage:
results = _search_stackoverflow_for_relevant_questions("python", "pytorch", 5)

In [30]:
print(format_stackoferflow_questions(results))

---------------
Question:
How do I check if PyTorch is using the GPU? The `nvidia-smi` command can detect GPU activity, but I want to check it directly from inside a Python script.
--------------- 
Accepted Answer:
These functions should help:



```
>>> import torch

>>> torch.cuda.is_available()
True

>>> torch.cuda.device_count()
1

>>> torch.cuda.current_device()
0

>>> torch.cuda.device(0)

>>> torch.cuda.get_device_name(0)
'GeForce GTX 950M'

```

This tells us:


* CUDA is available and can be used by one device.
* `Device 0` refers to the GPU `GeForce GTX 950M`, and it is currently chosen by PyTorch.---------------
Question:
How do I save a trained model in PyTorch? I have read that:


1. `torch.save()`/`torch.load()` is for saving/loading a serializable object.
2. `model.state_dict()`/`model.load_state_dict()` is for saving/loading model state.
--------------- 
Accepted Answer:
Found this page on their github repo:



#### Recommended approach for saving a model


There are tw

In [26]:
for result in results[0:2]:
    print(f"---------------\nQuestion:\n{result[0]}\n--------------- \nAccepted Answer:\n{result[1]}")

---------------
Question:
How do I check if PyTorch is using the GPU? The `nvidia-smi` command can detect GPU activity, but I want to check it directly from inside a Python script.
--------------- 
Accepted Answer:
These functions should help:



```
>>> import torch

>>> torch.cuda.is_available()
True

>>> torch.cuda.device_count()
1

>>> torch.cuda.current_device()
0

>>> torch.cuda.device(0)

>>> torch.cuda.get_device_name(0)
'GeForce GTX 950M'

```

This tells us:


* CUDA is available and can be used by one device.
* `Device 0` refers to the GPU `GeForce GTX 950M`, and it is currently chosen by PyTorch.
---------------
Question:
How do I save a trained model in PyTorch? I have read that:


1. `torch.save()`/`torch.load()` is for saving/loading a serializable object.
2. `model.state_dict()`/`model.load_state_dict()` is for saving/loading model state.
--------------- 
Accepted Answer:
Found this page on their github repo:



#### Recommended approach for saving a model


There are t

In [33]:
questions_and_answers = _extract_questions_and_answers(answers)

In [34]:
questions_and_answers

[{'title': 'if/else in a list comprehension',
  'accepted_answer': 'No answer body found.'},
 {'title': 'List comprehension vs. lambda + filter',
  'accepted_answer': 'No answer body found.'},
 {'title': 'List comprehension vs map',
  'accepted_answer': 'No answer body found.'}]